In [1]:
import pandas as pd
import csv 
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
import HTMLParser
from nltk import word_tokenize
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import pickle
from sklearn import svm, linear_model, naive_bayes 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from sklearn.model_selection import train_test_split

X = pd.read_csv("X_train.csv")['0']
final_test = pd.read_csv("X_test.csv")['0']
Y = pd.read_csv("Y_train.csv")['0']
print 'read X, Y, final_test done'
#type(X)

from collections import Counter
Counter(Y)

read X, Y, final_test done


Counter({'hockey': 13994,
         'movies': 14847,
         'nba': 12325,
         'news': 13986,
         'nfl': 13392,
         'politics': 13205,
         'soccer': 14224,
         'worldnews': 14027})

In [2]:
def count_vec(X_train,X_test):
    print "Using Count Vectorizer\n"
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    X_test_counts = count_vect.transform(X_test)
    return X_train_counts, X_test_counts

# this function sort the training dataset into several smaller datasets 
# each smaller dataset has the same class label
def sortByLabels(X_train, Y_train):
    X=[]
    Y=[]
    print "start to sort by labels"
    for i in set(Y_train):
        X.append(X_train[np.asarray(Y_train)==i,])
        Y.append(i)
    return X,Y

# this function counts the average frequency of a word across all documents in the same class
def avg_frequecy(X):
    # X needs to be a matrix (2d array)
    #return only a vector for all documents in one class
    freq = X.sum(axis=0).astype('float')/X.shape[0]
    #print 'freq',freq
    return freq

# this function counts the the average frequency of each word in each class
def ave_freq_for_all_classes(X):
    print 'start to count average frequency for each word in all classes'
    # X is a list of matries (arrays) sorted by the labels
    f_matrix = []
    for i in range(len(X)):
        print(X[i].shape)
        f_matrix.append(np.squeeze(np.asarray(avg_frequecy(X[i]))))
        #print 'f_matrix shape',f_matrix[0]
    print np.matrix(f_matrix)
    return np.matrix(f_matrix)
#f_m = ave_freq_for_all_classes(irisY)

# this function finds the domain specific words for each class
def dsw(f_m,alpha):
    # input: the matrix of frequency for all words in each class
    print 'start to construct dsw matrix'
    x = np.zeros(f_m.shape)
    #print 'f_m',f_m.shape()
    for i in range(f_m.shape[0]):
        for j in range(f_m.shape[1]):
            sums = f_m[np.arange(len(f_m))!=i,j].sum()
            if f_m[i,j]> alpha*sums:
            # then this is a domain specific classifier
                x[i,j] = 1
    return x
#dsw(f_m,0.2)

# this function fits the training data to find the domain specific words matrix
def dec_fit(X_train_c,X_test_c,Y_train,alpha):
    #X_train_c, X_test_c = count_vec(X_train,X_test)
    X_c_labels, Y_c_labels = sortByLabels(X_train_c, Y_train)
    #print 'X_c_labels',X_c_labels[0]
    f_m = ave_freq_for_all_classes(X_c_labels)
    dsw_matrix = dsw(f_m,alpha)
    return dsw_matrix, Y_c_labels, X_test_c

####this is a sperate two functions for dsw fiting fir easier hyper-parameter setting 
# this function find the average frequency matrix for the training data
def f_m_fit(X_train_c,X_test_c,Y_train):
    #X_train_c, X_test_c = count_vec(X_train,X_test)
    X_c_labels, Y_c_labels = sortByLabels(X_train_c, Y_train)
    f_m = ave_freq_for_all_classes(X_c_labels)
    return f_m, Y_c_labels, X_test_c

#
def dec_fit1(f_m,alpha):
    dsw_matrix = dsw(f_m,alpha)
    for i in range(len(dsw_matrix)):
        print 'domain specific words for class',i,'is',dsw_matrix[i].sum()
    return dsw_matrix

def dsc_pred(X_test_c,Y_c_labels, dsw_m):
    pred = []
    for i in range(X_test_c.shape[0]):
        compare = np.ones(len(Y_c_labels))
        for j in range(len(Y_c_labels)):
            #print compare[j]
            compare[j] = np.inner(X_test_c[i].toarray(), dsw_m[j]).astype('float')/dsw_m[j].sum()
        #print(compare)  
        index = compare.argmax()
        pred.append(Y_c_labels[index]) 
    return pred


In [3]:
def pipeline_dsw(f_m, alpha, X_test_c, Y_c_labels, y_test):
    dsw_m = dec_fit1(f_m,alpha)
    predicted = dsc_pred(X_test_c,Y_c_labels, dsw_m)
    from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
    print 'overall accuracy', accuracy_score(y_test, predicted)
    print classification_report(y_test, predicted)  

In [7]:
# test on validation set in single fold or 10 folds

# a single split
splits = int(X.shape[0]*0.9)

X_train_val, X_test_val = X[:splits,], X[splits:,]
Y_train_val, Y_test_val = Y[:splits,], Y[splits:,]

X_train_count, X_test_count = count_vec(X_train_val, X_test_val)
f_m, Y_c_labels, X_test_c = f_m_fit(X_train_count,X_test_count,Y_train_val)

for alpha in np.arange(0.1,3,0.3):
    print '##########################'
    print 'this is using alpha=', alpha
    
    pipeline_dsw(f_m, alpha, X_test_c,Y_c_labels, Y_test_val)


Using Count Vectorizer

start to sort by labels
start to count average frequency for each word in all classes
(12053, 85624)
(13372, 85624)
(12608, 85624)
(12589, 85624)
(12645, 85624)
(11860, 85624)
(11071, 85624)
(12802, 85624)
[[  4.14834481e-04   0.00000000e+00   5.80768273e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.24349387e-04   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    7.47831289e-05   0.00000000e+00]
 [  7.93147208e-05   7.13832487e-04   0.00000000e+00 ...,   7.93147208e-05
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  8.43170320e-05   2.52951096e-04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.61304309e-04   2.70978231e-04   9.03260771e-05 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.81127949e-05   1.56225590e-04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
##########################
this is using alpha= 0.1
start to construct dsw matrix
domain s

In [17]:
from time import time
##validation set
#X_train_c, X_test_c, y_train, y_test = split_test_train_data(X_train_count,y_train,0.8)
X_train_count, X_test_count = count_vec(X, final_test)

start_time = time()
f_m, Y_c_labels, X_test_c = f_m_fit(X_train_count,X_test_count,Y)
dsw_m = dec_fit1(f_m,1.4)
predicted_final_test = dsc_pred(X_test_c,Y_c_labels, dsw_m)
print 'run time',time()-start_time

Using Count Vectorizer

start to sort by labels
start to count average frequency for each word in all classes
(13392, 89524)
(14847, 89524)
(13994, 89524)
(13986, 89524)
(14027, 89524)
(13205, 89524)
(12325, 89524)
(14224, 89524)
[[  3.73357228e-04   0.00000000e+00   5.22700119e-04 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.02061022e-04   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    6.73536741e-05   0.00000000e+00]
 [  7.14591968e-05   6.43132771e-04   0.00000000e+00 ...,   7.14591968e-05
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  7.57288906e-05   2.27186672e-04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.24543611e-04   2.43407708e-04   8.11359026e-05 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  7.03037120e-05   1.40607424e-04   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
start to construct dsw matrix
domain specific words for class 0 is 6398.0
domain specific 

In [ ]:
X_train_count, X_test_count = count_vec(X, final_test)
f_m, Y_c_labels, X_test_c = f_m_fit(X_train_count,X_test_count,Y)
dsw_m = dec_fit1(f_m,1.4)
predicted_final_test = dsc_pred(X_test_c,Y_c_labels, dsw_m)

In [ ]:
print "Predictions on test file........................"
sys.stdout.flush()

#predicted_final_test = sgd_clf.predict(testX)
predict_file = pd.DataFrame(columns=['id','category'])
predict_file.id= range(X_test_count.shape[0])
predict_file.category = predicted_final_test

predict_file.to_csv('predictions/Y_test_DSC.csv',index=False)
print "Done."